# Titanic幸存者预测
* ## 1、介绍
* ## 2、数据导入与检查
* ## 3、数据探索
* ## 4、数据清洗、转换、规约
* ## 5、建模

## 1、介绍
### 该项目是Kaggle上的入门项目，用于预测泰坦尼克号上，哪些乘客能幸存


## 2、数据导入与检查

* ### 2.1相关包导入

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')


* ### 2.2数据导入

In [ ]:
train = pd.read_csv('C:/Users/loukey/Desktop/RGL/Tatanic/data/train.csv')
test = pd.read_csv('C:/Users/loukey/Desktop/RGL/Tatanic/data/test.csv')

* ### 2.3数据检查

* #### 2.3.1异常值处理

In [ ]:
def detect_outliers(df,n,features):
    outlier_indexs=[]
    for col in features :
        Q1 = np.percentile(df[col],25)
        Q3 = np.percentile(df[col],75)
        IQR = Q3-Q1
        outlier_step = 1.5*IQR
        indexs = df[col][ (df[col]<Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        outlier_indexs.extend(indexs)
    indexs_counter = Counter(outlier_indexs)
    multi_indexs = list(k for k,v in indexs_counter.items() if v>n)
    return multi_indexs

outliers_drop=detect_outliers(train,2,['Age','SibSp','Parch','Fare'])
train.loc[outliers_drop]


In [ ]:
# dorp it!
train.drop(outliers_drop,axis=0).reset_index(drop=True)

* #### 2.3.2空值缺失值查看

In [ ]:
# join train and test set
dataset = pd.concat([train,test],axis=0).reset_index(drop=True)
dataset=dataset.fillna(np.NaN)

dataset.isnull().sum()


In [ ]:
dataset.describe()

In [ ]:
train.isnull().sum()

### 可见Age和Cabin的缺失度较高，考虑做填充处理

## 3、数据探索

* ### 3.1连续值分布

 &emsp; __heatmap__

In [ ]:
g = sns.heatmap(train[ ['Survived','Age','SibSp','Parch','Fare'] ].corr(),fmt='.2f',annot=True,cmap='coolwarm')

&emsp;与目标变量Survived相关性强的只有Fare，其他相关性低，但不代表不重要，因为相关系数只是衡量线性相关而已。  
&emsp;我们进一步探索其与目标变量的关系

&emsp;__SibSp__

In [ ]:
g = sns.factorplot(x='SibSp',y='Survived',data=train,kind='bar',size=6)
g.set_ylabels('survival probability')

&emsp;可以出随着兄弟姐妹的增加，生存几率在减小。  
&emsp;单身、1-2个同伴的生存概率更大。  
&emsp;由此，我们可以衍生出同伴人数的特征变量。


&emsp;__Parch__

In [ ]:
g = sns.factorplot(x='Parch',y='Survived',data=train,kind='bar',size=6)
g.set_ylabels('survived probability')

&emsp;与SibSp的分布类似，家庭规模越大，生存几率越小。  
&emsp;中等规模家庭（1-3）的生存几率最大，大型家庭（5-6人）生存几率最小。  
&emsp;可以衍生出家庭规模的特征变量，但是注意，3人家庭分布存在很大的方差。

&emsp;__Age__

In [ ]:
g = sns.distplot(train['Age'][ (train['Survived'] == 0) & (train['Age'].notnull()) ],color='Red',hist=False)
g = sns.distplot(train['Age'][ (train['Survived'] == 1) & (train['Age'].notnull()) ],color='Blue',hist=False)
g.set_xlabel('Age')
g.set_ylabel('Frequency')
g = g.legend(['Not Survived','Survived'])

&emsp;可以看出年龄大致呈高斯分布。  
&emsp;而且在Survived和非Survived中，两者有一定差异，尤其在0-5岁区间，Survived明显凸起，说明该年龄段的生存率偏高。  
&emsp;而70岁以上的老人明显生产记录较低，所以虽然Age不与Survived直接线性相关，但是不可否认其对目标变量的区分度。

&emsp;__Fare__

In [ ]:
#前面探索发现Fare有缺失值，画图时有缺失值会报错，故先用中位数填充空值
dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].median())
g = sns.distplot(dataset['Fare'],label='Skewness:%.2f'%(dataset['Fare'].skew()))
g = g.legend(loc='best')

&emsp;偏度过大，会使得某些值对模型影响过大，取对数平滑

In [ ]:
dataset['Fare'] = dataset['Fare'].map(lambda x:np.log(x) if x>0 else 0)
g = sns.distplot(dataset['Fare'],label='Skewness:%.2f'%(dataset['Fare'].skew()))
g.legend(loc='best')

&emsp;偏度下降很明显

* ### 3.2离散值分布

&emsp;__Sex__

In [ ]:
g = sns.factorplot(x='Sex',y='Survived',data=train,kind='bar')
g = g.set_ylabels('survival probability')

In [ ]:
train[['Survived','Sex']].groupby('Sex').mean()

&emsp;很明显，女性比男性的生存率高的多，女士优先原则！

&emsp;__Pclass__

In [ ]:
g = sns.factorplot(x='Pclass',y='Survived',data=train,kind='bar')
g = g.set_ylabels('survival probability')

In [ ]:
train[['Survived','Pclass']].groupby('Pclass').mean()

In [ ]:
g = sns.factorplot(x='Pclass',y='Survived',data=train,hue='Sex',kind='bar',size=4)
g.set_ylabels('survival probability')

&emsp;船舱等级越高，生存率越高，这对男女都一样。

&emsp;__Embarked__

In [ ]:
dataset['Embarked'].isnull().sum()

In [ ]:
dataset['Embarked'].describe()

In [ ]:
#用众数填充
dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [ ]:
g = sns.factorplot(x='Embarked',y='Survived',data=train,kind='bar')
g = g.set_ylabels('survived probability')

&emsp;分布没有明显区分度，但C处生存率比较高，猜测上车点跟船舱等级Pclass有关系，C处上船的人可能处于较高的船舱等级。

In [ ]:
g = sns.factorplot(x='Embarked',y='Survived',data=train,hue='Pclass',kind='bar')
g = g.set_ylabels('survived probability')

&emsp;在S和C处，头等舱的人数占比偏大，我们再看人数的分布。

In [ ]:
g = sns.factorplot(x='Pclass',col='Embarked',data=train,kind='count',size=6)
g = g.set_ylabels('Count')

&emsp;从S上船的人大多都坐的3等座，而Q处基本全是3等座，所以Q处上船的人相对来说生产几率更小

* ## 4、数据清洗、转换、规约

* ### 4.1数据清洗、转换

&emsp;前面得到小孩具有更大的生产几率，老人生产几率很低。  
&emsp;说明Age具有一定区分度，插值法有很多，如均值、中位数和众数，复杂一点的有拉格朗日插值法和牛顿插值法等。我们这里利用关联变量去预测年龄值。  
&emsp;用这几个关联特征：Name,Sex,SibSp,Parch，选择Name是因为其开头为Mr、Miss等可以框定年龄范围，故先对Name属性进行截取和离散化

&emsp;__Name__

In [ ]:
dataset['Name'].head()

In [ ]:
dataset['Title']=dataset['Name'].map(lambda x:x.split(',')[1].split('.')[0].strip(' '))

In [ ]:
g = sns.countplot(x='Title',data=dataset)
g = plt.setp(g.get_xticklabels(),rotation=45)

In [ ]:
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer',
                                             'Dona'], 'Rare')
dataset["Title"] = dataset["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
dataset["Title"] = dataset["Title"].astype(int)

In [ ]:
g = sns.countplot(dataset['Title'])
g = g.set_xticklabels(['Master','Miss/Ms/Mme/Mlle/Mrs','Mr','Rare'])

In [ ]:
g = sns.factorplot(x='Title',y='Survived',data=dataset,kind='bar')
g = g.set_xticklabels(['Master','Miss','Mr','Rare'])
g = g.set_ylabels('survival probability')

&emsp;小孩和女人优先！

In [ ]:
dataset.drop('Name',axis=1,inplace=True)

&emsp;__Age__

&emsp;用这几个关联特征：Title,Sex,Pclass,SibSp,Parch,

In [ ]:
g = sns.factorplot(x='Title',y='Age',data=dataset,kind='box')
g = sns.factorplot(x='Sex',y='Age',hue='Pclass',data=dataset,kind='box') #在性别上分布无差异，引入Pclass
g = sns.factorplot(x='Pclass',y='Age',data=dataset,kind='box')
g = sns.factorplot(x='SibSp',y='Age',data=dataset,kind='box')
g = sns.factorplot(x='Parch',y='Age',data=dataset,kind='box')

&emsp;性别对年龄无区分度。  
&emsp;船舱等级越高，年龄越大。  
&emsp;同伴越多越年轻，长辈/晚辈越多，年级越大。

In [ ]:
dataset['Sex'] = dataset['Sex'].map({'male':0,'female':1})

In [ ]:
g = sns.heatmap(dataset[['Age','Title','Sex','SibSp','Parch','Pclass']].corr(),cmap='coolwarm',annot=True)

&emsp;Sex基本不相关，其他都可用，我们用Title,SibSp,Parch,Pclass相似的人的年龄中位数来代替缺失值。

In [ ]:
age_nan_index = dataset['Age'].isnull().index
age_median = dataset['Age'].median()
for i in age_nan_index:    
    age_pred = dataset['Age'][((dataset.iloc[i]['Title'] == dataset['Title']) & (dataset.iloc[i]['SibSp'] == dataset['SibSp']) 
                              & (dataset.iloc[i]['Parch'] ==dataset['Parch']) & (dataset.iloc[i]['Pclass'] == dataset['Pclass']))].median()
    if not np.isnan(age_pred):
        dataset['Age'].iloc[i] = age_pred
    else:
        dataset['Age'].iloc[i] = age_median